## Description: prepare the notebook for manual annotation

In this notebook, three elements are added to the previously generated dataset:
- check that the chapter and verse_num columns are integers
- sign_info column, based on a recent version of the function
- reconstructed_verse column which is a gcons version of the verse with brackets indicating the reconstructed sections
- the spatial_arg_type column for annotating goal, source, location, etc. 
- remove the rows which have "MN" in preposition_1 and no preposition_2

In [1]:
import pandas as pd

from itertools import chain

from tfob import TFOb,  get_dss

DSS = get_dss()

PATH = 'data/biblical_datasets/ezekiel/annotation_df_history/ezekiel_dataset.csv' # Modify before executing the notebook

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [2]:
#df_1 = pd.read_csv('../2_datasets/generate_datasets/data/biblical_datasets/Jeremiah_dataset.csv').fillna("")
df = pd.read_csv(PATH).fillna("")

In [3]:
# Make the values in "chapter" and "verse_num" integers
df['chapter'] = df['chapter'].astype(int)
df['verse_num'] = df['verse_num'].astype(int)

In [4]:
# Reorganise the dataset by chapter, verse_num and scroll

# Custom sorting logic for "scroll" column
df['scroll_order'] = df['scroll'].apply(lambda x: 0 if x == "MT" else 1)

# Sort by "scroll_order", "chapter", and "verse_num"
df = df.sort_values(by=['chapter', 'verse_num', 'scroll_order'])

# Drop the temporary 'scroll_order' column
df = df.drop(columns='scroll_order')

df = df.reset_index(drop=True)

In [5]:
pd.set_option("display.max_columns", None)

In [15]:
df.head(20)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
0,264812,BW>[,MT,Ezekiel,1,4,B>H,W >R> W HNH RWX S<RH B>H MN H YPWN <NN GDWL W ...,RWX S<RH B>H MN H YPWN,RWX S<RH,MN H YPWN,MN H YPWN/,0.0,prep,,,det,subs,simple,,,qal,ptca,MN,,,,,,,,,,,
1,264898,SBB[,MT,Ezekiel,1,9,JSBW,XBRT >CH >L >XWTH KNPJHM L> JSBW B LKTN >JC >L...,L> JSBW,,no complement,no complement,,,,,,,,,,nif,impf,,,,,,,,,,,,
2,264900,HLK[,MT,Ezekiel,1,9,LKTN,XBRT >CH >L >XWTH KNPJHM L> JSBW B LKTN >JC >L...,B LKTN,,no complement,no complement,,,,,,,,,,qal,infc,,,,,,,,,,,,
3,264905,HLK[,MT,Ezekiel,1,9,JLKW,XBRT >CH >L >XWTH KNPJHM L> JSBW B LKTN >JC >L...,JLKW,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
4,2081818,HLK[,11Q4,Ezekiel,1,9,LKTM,XBRT >CH >L >XWTH KNPJHM L> JSBW B LKTM >JC >L...,None,None,,,,,,,,,,0.0,1111 111 11 11111 111111 11 1000 0 0011 111 11...,qal,infc,,,,,,,,,,,[XBRT >CH >L >XWTH KNPJHM L> J]SBW B LK[TM >JC...,
5,2081823,HLK[,11Q4,Ezekiel,1,9,JLKW,XBRT >CH >L >XWTH KNPJHM L> JSBW B LKTM >JC >L...,None,None,,,,,,,,,,0.0,1111 111 11 11111 111111 11 1000 0 0011 111 11...,qal,impf,,,,,,,,,,,[XBRT >CH >L >XWTH KNPJHM L> J]SBW B LK[TM >JC...,
6,264955,HLK[,MT,Ezekiel,1,12,JLKW,W >JC >L <BR PNJW JLKW >L >CR JHJH CMH H RWX L...,W JLKW,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
7,264963,HLK[,MT,Ezekiel,1,12,LKT,W >JC >L <BR PNJW JLKW >L >CR JHJH CMH H RWX L...,L LKT,,no complement,no complement,,,,,,,,,,qal,infc,,,,,,,,,,,,
8,264964,HLK[,MT,Ezekiel,1,12,JLKW,W >JC >L <BR PNJW JLKW >L >CR JHJH CMH H RWX L...,JLKW,,no complement,no complement,,,,,,,,,,qal,impf,,,,,,,,,,,,
9,264966,SBB[,MT,Ezekiel,1,12,JSBW,W >JC >L <BR PNJW JLKW >L >CR JHJH CMH H RWX L...,L> JSBW,,no complement,no complement,,,,,,,,,,nif,impf,,,,,,,,,,,,


In [7]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments'],
      dtype='object')

In [8]:
# This is the most recent sign_info function which uses the .rec (1 October 2024)

def sign_info(verb):
    sign_rec_words = []
    
    scroll = verb.to_scrolls.scroll[0]
    verse = TFOb.section([verb.book[0], verb.chapter[0], verb.verse[0]], DSS, scroll)
    
    for word in verse.to_words:
        signs = word.to_signs
        sign_rec = []
        for sign in signs:
            if sign.type[0] == "cons": # only keeps consonants
                if sign.rec[0]:
                    sign_rec.append("1")
                else:
                    sign_rec.append("0")
        if sign_rec:
            sign_rec_words.append("".join(sign_rec))
        
    return " ".join(sign_rec_words).strip()

In [9]:
# Updates the dataframe (change the old sign_info column to one containing the return of the new function)

def update_sign_info(row):
    if row['scroll'] != 'MT':
        return sign_info(TFOb(int(row['verb_id']), DSS))
    return row['sign_info']

# Apply the function to the dataframe
df['sign_info'] = df.apply(update_sign_info, axis=1)

In [10]:
# Initialize an empty list to collect mismatch information
mismatches = []

# Function to check the lengths of sign_info and gcons_verse
def check_length(row):
    if row['scroll'] != 'MT':
        if len(row['sign_info']) != len(row['gcons_verse']):
            mismatches.append(f"Mismatch found in row {row.name}: sign_info='{row['sign_info']}' (len={len(row['sign_info'])}), gcons_verse='{row['gcons_verse']}' (len={len(row['gcons_verse'])})")

# Apply the function to the dataframe
df.apply(check_length, axis=1)

# Check if there are any mismatches and print them
if mismatches:
    print("Mismatches found:")
    for mismatch in mismatches:
        print(mismatch)
else:
    print("No mismatches found.")

No mismatches found.


In [11]:
# The reconstruct_verse function adds a gcons version of the verse with brackets indicating the reconstructed sections

# Add a column with the verse plus brackets indicating reconstructed letters

def reconstruct_verse(sign_info, verse):
    rec_verse = []
    is_reconstructed = False

    for c_verse, c_sign in zip(verse, sign_info):
        if c_sign == "1" and not is_reconstructed:
            is_reconstructed = True
            rec_verse.append("[")
        if c_sign == "0" and is_reconstructed:
            is_reconstructed = False
            if rec_verse[-1] == " ":
                rec_verse.pop()
                rec_verse.extend(["]", " "])
            else:
                rec_verse.append("]")
        rec_verse.append(c_verse)

    if is_reconstructed:
        rec_verse.append("]")
    return "".join(rec_verse)


df["reconstructed_verse"] = ""

df["reconstructed_verse"] = df.apply(lambda row: reconstruct_verse(row.sign_info, row.gcons_verse) if row.book != "MT" else "", axis=1)


In [12]:
# Delete the DSS duplicated

verb_ids = df.verb_id.to_list()

duplicates = set([verb_id for verb_id in verb_ids if verb_ids.count(verb_id) > 1])


df = df[~((df.gcons_clause == "None") & (df.subject == "None") & (df.verb_id.isin(duplicates)))] # delete duplicates
df.reset_index(drop=True, inplace=True) # reset the index for AnnotEasy

In [13]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse'],
      dtype='object')

In [14]:
# Add the spatial_arg_type column

df["spatial_arg_type"] = ""

In [16]:
df.columns

Index(['verb_id', 'lex', 'scroll', 'book', 'chapter', 'verse_num',
       'gcons_verb', 'gcons_verse', 'gcons_clause', 'subject', 'complement',
       'cmpl_lex', 'dir_he', 'cmpl_constr', 'cmpl_nt', 'cmpl_anim', 'cmpl_det',
       'cmpl_indiv', 'cmpl_complex', 'dir_he_dss', 'sign_info', 'stem',
       'tense', 'preposition_1', 'preposition_2', 'preposition_3',
       'preposition_4', 'preposition_5', 'preposition_6', 'preposition_7',
       'cmpl_translation', 'motion_type', 'comments', 'reconstructed_verse',
       'spatial_arg_type'],
      dtype='object')

In [17]:
# Remove the columns that have MN as preposition 1 and not other prepositions

In [18]:
# have a look at the data
df_test = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]
df_test.head(5)
#len(df_test)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type
0,264812,BW>[,MT,Ezekiel,1,4,B>H,W >R> W HNH RWX S<RH B>H MN H YPWN <NN GDWL W ...,RWX S<RH B>H MN H YPWN,RWX S<RH,MN H YPWN,MN H YPWN/,0.0,prep,,,det,subs,simple,,,qal,ptca,MN,,,,,,,,,,,
16,264996,JY>[,MT,Ezekiel,1,13,JWY>,W DMWT H XJWT MR>JHM K GXLJ >C B<RWT K MR>H H ...,W MN H >C JWY> BRQ,BRQ,MN H >C,MN H >C/,0.0,prep,,,det,subs,simple,,,qal,ptca,MN,,,,,,,,,,,
49,265893,CWB[,MT,Ezekiel,3,20,CWB,W B CWB YDJQ M YDQW W <FH <WL W NTTJ MKCWL L P...,B CWB YDJQ M YDQW,YDJQ,M YDQW,MN YDQ/,0.0,prep,,,det,subs,simple,,,qal,infc,MN,,,,,,,,,,,
59,266519,JY>[,MT,Ezekiel,5,4,TY>,W MHM <WD TQX W HCLKT >WTM >L TWK H >C W FRPT ...,MMNW TY> >C >L KL BJT JFR>L,>C,MMNW,MN,0.0,prep,,,det,,simple,,,qal,impf,MN,,,,,,,,,,,
92,267525,SBB[,MT,Ezekiel,7,22,HSBWTJ,W HSBWTJ PNJ MHM W XLLW >T YPWNJ W B>W BH PRJY...,W HSBWTJ PNJ MHM,,MHM,MN,0.0,prep,,,det,,simple,,,hif,perf,MN,,,,,,,,,,,


In [19]:
# Identify the rows to drop based on the condition
index_to_drop = df[(df.preposition_1 == "MN") & (df.preposition_2 == "")].index

# Drop the rows by index
df = df.drop(index_to_drop)

# Reset the index
df = df.reset_index(drop=True)

In [20]:
# Verification
df[(df.preposition_1 == "MN") & (df.preposition_2 == "")]

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,gcons_clause,subject,complement,cmpl_lex,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,dir_he_dss,sign_info,stem,tense,preposition_1,preposition_2,preposition_3,preposition_4,preposition_5,preposition_6,preposition_7,cmpl_translation,motion_type,comments,reconstructed_verse,spatial_arg_type


In [21]:
#df.head(20)

In [22]:
df.to_csv(PATH, index=False)